In [ ]:
# Train the best performing model. With Hourly Features and Log of Count
# Kaggle Score: 0.42658
import numpy as np
import pandas as pd
import os

# Define IAM role
import boto3
import re

import sagemaker

In [ ]:
# use a different profile
boto_session = boto3.Session(profile_name='ml_user_predict',region_name='us-east-1')

In [ ]:
sess = sagemaker.Session(boto_session=boto_session)

In [ ]:
endpoint_name = 'xgboost-biketrain-v1'

predictor = sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name,
                                                 sagemaker_session=sess)

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

In [ ]:
def run_predictions(arr_input, batch_size):
    predictions = []
    
    if type(arr_input) != np.ndarray:
        arr_input = np.array(arr_input)
        
    for arr in np.array_split(arr_input,batch_size):
        if arr.size > 0: 
            print("Shape:{0}".format(arr.shape))
            result = predictor.predict(arr)
            result = result.decode("utf-8")
            result = result.split(',')
            predictions += [np.expm1(float(r)) for r in result]
    
    return predictions

In [ ]:
df_test = pd.read_csv('bike_test.csv')

In [ ]:
df_test.head()

In [ ]:
tmp_df = df_test.as_matrix(columns=['season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek','hour'])

In [ ]:
tmp_df.shape

In [ ]:
predicted_count = run_predictions(arr_input=tmp_df, batch_size=10)

In [ ]:
predicted_count[:5]

In [ ]:
df_test['count'] = predicted_count

In [ ]:
df_test.head()

In [ ]:
df_test['count'].describe()